In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from tqdm import tqdm

from scipy.spatial import ConvexHull

import lsst.daf.butler as dafButler


from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.visualization import ZScaleInterval, SqrtStretch, ImageNormalize, ManualInterval, AsinhStretch, MinMaxInterval, LogStretch


import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from typing import TYPE_CHECKING, cast

import astropy.units as u
from astropy.coordinates import SkyCoord

plt.set_loglevel('WARNING')

from tqdm import tqdm
import pandas as pd


INFO:numexpr.utils:Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.


In [8]:
# Load collection
repo = '/repo/main'
instrument = "LSSTComCam"

butler_11= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/w_2025_11/DM-49472', instrument=instrument)
butler_12= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/d_2025_03_25/DM-49693', instrument=instrument)
butler_13= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751', instrument=instrument)
butler_14= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864', instrument=instrument)

test_11 = butler_11.getURI('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
test_12 = butler_11.getURI('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
test_13 = butler_13.getURI('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
test_14 = butler_14.getURI('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
test_14_dia = butler_14.getURI('dia_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
obj_ref = butler_14.query_datasets("object")
test_11_obj_frc_src = butler_11.get('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
test_14_obj_frc_src = butler_14.get('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})

test_14_obj = butler_14.getURI('object',  dataId=obj_ref[10].dataId)
print(test_11)
print(test_12)
print(test_13)
print(test_14)

size_mb = test_11.size() / (1024 ** 2)
print(f"Size in w11: {size_mb:.2f} MB")

size_mb = test_12.size() / (1024 ** 2)
print(f"Size in w12: {size_mb:.2f} MB")

size_mb = test_13.size() / (1024 ** 2)
print(f"Size in w13: {size_mb:.2f} MB")

size_mb = test_14.size() / (1024 ** 2)
print(f"Size in w14: {size_mb:.2f} MB")

file:///sdf/group/rubin/repo/main/LSSTComCam/runs/DRP/DP1/w_2025_11/DM-49472/20250317T052612Z/object_forced_source/10704/22/object_forced_source_10704_22_lsst_cells_v1_LSSTComCam_runs_DRP_DP1_w_2025_11_DM-49472_20250317T052612Z.parq
file:///sdf/group/rubin/repo/main/LSSTComCam/runs/DRP/DP1/w_2025_11/DM-49472/20250317T052612Z/object_forced_source/10704/22/object_forced_source_10704_22_lsst_cells_v1_LSSTComCam_runs_DRP_DP1_w_2025_11_DM-49472_20250317T052612Z.parq
file:///sdf/group/rubin/repo/main/LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751/20250331T004753Z/object_forced_source/10704/22/object_forced_source_10704_22_lsst_cells_v1_LSSTComCam_runs_DRP_DP1_w_2025_13_DM-49751_20250331T004753Z.parq
file:///sdf/group/rubin/repo/main/LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864/20250406T123755Z/object_forced_source/10704/22/object_forced_source_10704_22_lsst_cells_v1_LSSTComCam_runs_DRP_DP1_w_2025_14_DM-49864_20250406T123755Z.parq
Size in w11: 18.38 MB
Size in w12: 18.38 MB
Size in w13: 3.10 MB
Siz

In [16]:
test_11 = butler_11.getURI('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
test_14 = butler_14.getURI('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
test_11_obj_frc_src = butler_11.get('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})
test_14_obj_frc_src = butler_14.get('object_forced_source', dataId={'skymap':'lsst_cells_v1', 'tract': 10704, 'patch': 22})

print(test_11.size()/ len(test_11_obj_frc_src))
print(test_14.size()/ len(test_14_obj_frc_src))

106.06214120198368
37.3109433098793


106.06214120198368
37.3109433098793


In [7]:
3.11 *10**6/ 87405

35.58148847319947

In [9]:
len(test_11_obj_frc_src)

181683

In [6]:
test_14_obj_frc_src

,objectId,parentObjectId,coord_ra,coord_dec,visit,detector,band,psfFlux,psfFluxErr,psfFlux_flag,...,pixelFlags_interpolatedCenter,pixelFlags_nodata,pixelFlags_saturated,pixelFlags_saturatedCenter,pixelFlags_suspect,pixelFlags_suspectCenter,invalidPsfFlag,tract,patch,forcedSourceId
0,4707765742508966102,0,37.947271,7.681758,2024112300212,0,r,115.277351,204.957581,False,...,False,False,False,False,False,False,False,10704,22,191403211796519290
1,4707765742508966103,0,37.905542,7.681783,2024112300212,0,r,335.256287,205.696945,False,...,False,False,False,False,False,False,False,10704,22,191403211796519291
2,4707765742508966111,0,37.938540,7.682122,2024112300212,0,r,154.461533,205.282745,False,...,False,False,False,False,False,False,False,10704,22,191403211796519292
3,4707765742508966120,0,37.942480,7.682745,2024112300212,0,r,316.116364,204.903656,False,...,False,False,False,False,False,False,False,10704,22,191403211796519293
4,4707765742508966124,0,37.936201,7.683035,2024112300212,0,r,59.660488,205.262421,False,...,False,False,False,False,False,False,False,10704,22,191403211796519295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87400,4707765742508978966,0,37.890982,7.703873,2024120700185,2,r,330.592834,259.074280,False,...,False,False,False,False,False,False,False,10704,22,191895764023132755
87401,4707765742508979088,0,37.857219,7.742143,2024120700185,2,r,270.035278,257.800873,False,...,False,False,False,False,False,False,False,10704,22,191895764023132809
87402,4707765742508979089,0,37.857523,7.742316,2024120700185,2,r,73.969772,257.312561,False,...,False,False,False,False,False,False,False,10704,22,191895764023132810
87403,4707765742508979121,0,37.850040,7.756606,2024120700185,2,r,-670.642883,249.939758,True,...,True,False,False,False,False,False,False,10704,22,191895764023132827


In [5]:
import pyarrow.parquet as pq

parquet_file = pq.ParquetFile(str(test_14)[7:])
dia_parquet_file = pq.ParquetFile(str(test_14_dia)[7:])
obj_ref_parquet_file = pq.ParquetFile(str(test_14_obj)[7:])

# Get number of row groups
num_row_groups = parquet_file.num_row_groups

print(f"Number of row groups: {num_row_groups}")


Number of row groups: 1


In [4]:
dia_parquet_file

In [5]:
lc_refs = butler_14.query_datasets("source2")
dia_lc_refs = butler_14.query_datasets("dia_source")

In [6]:
test_13 = butler_13.getURI('source2', dataId=lc_refs[0].dataId)
test_13

ResourcePath("file:///sdf/group/rubin/repo/main/LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751/20250331T004753Z/source2/20241108/i/i_06/2024110800245/source2_LSSTComCam_i_i_06_2024110800245_LSSTComCam_runs_DRP_DP1_w_2025_13_DM-49751_20250331T004753Z.parq")

In [7]:
lc_refs[0].dataId

{instrument: 'LSSTComCam', visit: 2024110800245, band: 'i', day_obs: 20241108, physical_filter: 'i_06'}

In [8]:
source2 = butler_14.get('source2', dataId={'instrument': 'LSSTComCam', 'visit': 2024110800245, 'band': 'i', 'day_obs': 20241108, 'physical_filter': 'i_06'})
source2.info()

<Table length=29537>
                 name                   dtype     class     n_bad
-------------------------------------- ------- ------------ -----
                              coord_ra float64       Column     0
                             coord_dec float64       Column     0
                        parentSourceId   int64       Column     0
                                     x float64       Column     0
                                     y float64       Column     0
                                  xErr float32 MaskedColumn   842
                                  yErr float32 MaskedColumn   842
                                    ra float64       Column     0
                                   dec float64       Column     0
                                 raErr float32 MaskedColumn   842
                                decErr float32 MaskedColumn   842
                            ra_dec_Cov float32 MaskedColumn   842
                             calibFlux float64 MaskedCo

In [40]:
dia_source = butler_14.get('dia_source', dataId=dia_lc_refs[0].dataId)  
dia_source.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239301 entries, 191473597720559653 to 191931215782084682
Data columns (total 86 columns):
 #   Column                              Non-Null Count   Dtype         
---  ------                              --------------   -----         
 0   visit                               239301 non-null  int64         
 1   detector                            239301 non-null  int16         
 2   band                                239301 non-null  object        
 3   diaObjectId                         239301 non-null  int64         
 4   ssObjectId                          239301 non-null  int64         
 5   parentDiaSourceId                   239301 non-null  int64         
 6   midpointMjdTai                      239301 non-null  float64       
 7   bboxSize                            239301 non-null  int64         
 8   time_processed                      239301 non-null  datetime64[ns]
 9   ra                                  239301 non-null  floa

In [50]:
dia_parquet_file

In [51]:
# Load the Parquet file
df = dia_source

# Columns you want to exclude from conversion
exclude = {'x', 'y', 'ra', 'dec', 'coord_ra', 'coord_dec'}

# Identify float64 columns (except the excluded ones)
float64_cols = [col for col in df.select_dtypes(include=['float64']).columns if col not in exclude]

# Convert those columns to float32
df[float64_cols] = df[float64_cols].astype('float32')

# Save back to Parquet
df.to_parquet('converted_file.parq', index=False)

In [52]:
converted_file.size

NameError: name 'converted_file' is not defined

In [48]:
rg = 0
obj_lc_parquet_file = dia_parquet_file
obj_lc_cols = obj_lc_parquet_file.metadata.num_columns
obj_lc_sizes = np.zeros(obj_lc_cols)
row_groups = obj_lc_parquet_file.metadata.num_row_groups
obj_lc_lens = np.zeros(obj_lc_cols)

for rg in range(row_groups):
    for col in range(obj_lc_cols):
        obj_lc_sizes[col] += (
            obj_lc_parquet_file.metadata.row_group(rg).column(col).total_compressed_size
        )
        obj_lc_lens[col] += (
            obj_lc_parquet_file.metadata.row_group(rg).column(col).num_values
        )

first_row_group = obj_lc_parquet_file.metadata.row_group(0)

column_names = [
    first_row_group.column(col).path_in_schema.removesuffix(".list.element")
    for col in range(0, first_row_group.num_columns)
]

column_types = [
    first_row_group.column(col).physical_type
    for col in range(0, first_row_group.num_columns)
]
obj_lc_frame = pd.DataFrame({"name": column_names,
                             "size": obj_lc_sizes, 
                             "length": obj_lc_lens,
                            "type": column_types})
obj_lc_frame["percent"] = obj_lc_frame["size"] / obj_lc_frame["size"].sum() * 100
obj_lc_frame["density"] = obj_lc_frame["size"] / obj_lc_frame["length"]
obj_lc_frame = obj_lc_frame.sort_values("density", ascending=False)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", 200)
obj_lc_frame

,name,size,length,type,percent,density
45,ixxPSF,152507.0,15626.0,DOUBLE,4.063505,9.759823
47,ixyPSF,152507.0,15626.0,DOUBLE,4.063505,9.759823
46,iyyPSF,152507.0,15626.0,DOUBLE,4.063505,9.759823
10,dec,152461.0,15626.0,DOUBLE,4.062279,9.756880
14,x,151035.0,15626.0,DOUBLE,4.024284,9.665621
15,y,150830.0,15626.0,DOUBLE,4.018822,9.652502
9,ra,147690.0,15626.0,DOUBLE,3.935157,9.451555
20,snr,119864.0,15626.0,DOUBLE,3.193742,7.670805
42,ixx,104552.0,15626.0,DOUBLE,2.785758,6.690900
43,iyy,104552.0,15626.0,DOUBLE,2.785758,6.690900


In [56]:
file_path = 'converted_file.parq'
size_bytes = os.path.getsize(file_path)
size_mb = size_bytes / (1024 * 1024)

print(f"Size of file: {size_mb:.2f} MB")

Size of file: 41.68 MB


In [58]:
test_14_dia = butler_14.getURI('dia_source',  dataId=dia_lc_refs[0].dataId)
size_mb = test_14_dia.size() / (1024 ** 2)
print(f"Size in w14: {size_mb:.2f} MB")


Size in w14: 50.33 MB


In [6]:
rg = 0
obj_lc_parquet_file = obj_ref_parquet_file
obj_lc_cols = obj_lc_parquet_file.metadata.num_columns
obj_lc_sizes = np.zeros(obj_lc_cols)
row_groups = obj_lc_parquet_file.metadata.num_row_groups
obj_lc_lens = np.zeros(obj_lc_cols)

for rg in range(row_groups):
    for col in range(obj_lc_cols):
        obj_lc_sizes[col] += (
            obj_lc_parquet_file.metadata.row_group(rg).column(col).total_compressed_size
        )
        obj_lc_lens[col] += (
            obj_lc_parquet_file.metadata.row_group(rg).column(col).num_values
        )

first_row_group = obj_lc_parquet_file.metadata.row_group(0)

column_names = [
    first_row_group.column(col).path_in_schema.removesuffix(".list.element")
    for col in range(0, first_row_group.num_columns)
]

column_types = [
    first_row_group.column(col).physical_type
    for col in range(0, first_row_group.num_columns)
]
obj_lc_frame = pd.DataFrame({"name": column_names,
                             "size": obj_lc_sizes, 
                             "length": obj_lc_lens,
                            "type": column_types})
obj_lc_frame["percent"] = obj_lc_frame["size"] / obj_lc_frame["size"].sum() * 100
obj_lc_frame["density"] = obj_lc_frame["size"] / obj_lc_frame["length"]
obj_lc_frame = obj_lc_frame.sort_values("density", ascending=False)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", 200)
obj_lc_frame

,name,size,length,type,percent,density
2,i_ra,236344.0,23916.0,DOUBLE,0.380870,9.882255
1317,coord_ra,236344.0,23916.0,DOUBLE,0.380870,9.882255
439,g_dec,236340.0,23916.0,DOUBLE,0.380864,9.882087
438,g_ra,236340.0,23916.0,DOUBLE,0.380864,9.882087
656,r_ra,236340.0,23916.0,DOUBLE,0.380864,9.882087
657,r_dec,236340.0,23916.0,DOUBLE,0.380864,9.882087
1318,coord_dec,236340.0,23916.0,DOUBLE,0.380864,9.882087
3,i_dec,236340.0,23916.0,DOUBLE,0.380864,9.882087
1354,sersic_ra,236222.0,23916.0,DOUBLE,0.380673,9.877153
1355,sersic_dec,236222.0,23916.0,DOUBLE,0.380673,9.877153


In [7]:
cols_per_band = []
for band in list("ugrizy"):
    for flux_type in ["psf", "kron"]:
        prefix = f"{band}_{flux_type}"
        cols_per_band.extend([f"{prefix}Flux", f"{prefix}FluxErr"])
    cols_per_band.append(f"{band}_kronRad")

obj_default_columns = [
    "objectId",
    "refFwhm",
    "shape_flag",
    "sky_object",
    "parentObjectId",
    "x",
    "y",
    "xErr",
    "yErr",
    "shape_yy",
    "shape_xx",
    "shape_xy",
    "coord_ra",
    "coord_dec",
    "coord_raErr",
    "coord_decErr",
    "tract",
    "patch",
    "detect_isIsolated",
] + cols_per_band

In [8]:
obj_default_columns

['objectId',
 'refFwhm',
 'shape_flag',
 'sky_object',
 'parentObjectId',
 'x',
 'y',
 'xErr',
 'yErr',
 'shape_yy',
 'shape_xx',
 'shape_xy',
 'coord_ra',
 'coord_dec',
 'coord_raErr',
 'coord_decErr',
 'tract',
 'patch',
 'detect_isIsolated',
 'u_psfFlux',
 'u_psfFluxErr',
 'u_kronFlux',
 'u_kronFluxErr',
 'u_kronRad',
 'g_psfFlux',
 'g_psfFluxErr',
 'g_kronFlux',
 'g_kronFluxErr',
 'g_kronRad',
 'r_psfFlux',
 'r_psfFluxErr',
 'r_kronFlux',
 'r_kronFluxErr',
 'r_kronRad',
 'i_psfFlux',
 'i_psfFluxErr',
 'i_kronFlux',
 'i_kronFluxErr',
 'i_kronRad',
 'z_psfFlux',
 'z_psfFluxErr',
 'z_kronFlux',
 'z_kronFluxErr',
 'z_kronRad',
 'y_psfFlux',
 'y_psfFluxErr',
 'y_kronFlux',
 'y_kronFluxErr',
 'y_kronRad']

In [17]:
filtered_frame = obj_lc_frame[obj_lc_frame["name"].isin(obj_default_columns)]
filtered_frame['percent'].sum()

np.float64(7.322996912997318)

In [18]:
obj_lc_frame['percent'].sum()

np.float64(100.00000000000001)

In [19]:
100/7.3

13.698630136986301

In [12]:
selected_columns = [col for col in obj_default_columns if col in obj_lc_frame.columns]
filtered_frame = obj_lc_frame[selected_columns]

In [13]:
filtered_frame

""
2
1317
439
438
656
657
1318
3
1354
1355
